In [14]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
(xtr, ytr), (xt, yt) = tf.keras.datasets.mnist.load_data()

In [4]:
xtr = np.reshape(xtr, (60000, 28 * 28))
xt = np.reshape(xt, (10000, 28 *28))

In [5]:
xtr = xtr/255
xt = xt/255

In [6]:
xt.shape

(10000, 784)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='sigmoid', input_shape=(784,)),
    tf.keras.layers.Dense(32, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
_ = model.fit(
    xtr, ytr,
    validation_data=(xt, yt),
    epochs=20, batch_size=2048,
    verbose=2
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 - 0s - loss: 0.2049 - accuracy: 0.9453 - val_loss: 0.2138 - val_accuracy: 0.9392
Epoch 2/20
60000/60000 - 0s - loss: 0.2007 - accuracy: 0.9468 - val_loss: 0.2105 - val_accuracy: 0.9403
Epoch 3/20
60000/60000 - 0s - loss: 0.1973 - accuracy: 0.9474 - val_loss: 0.2068 - val_accuracy: 0.9414
Epoch 4/20
60000/60000 - 0s - loss: 0.1933 - accuracy: 0.9485 - val_loss: 0.2036 - val_accuracy: 0.9424
Epoch 5/20
60000/60000 - 1s - loss: 0.1898 - accuracy: 0.9487 - val_loss: 0.2005 - val_accuracy: 0.9436
Epoch 6/20
60000/60000 - 1s - loss: 0.1863 - accuracy: 0.9498 - val_loss: 0.1972 - val_accuracy: 0.9437
Epoch 7/20
60000/60000 - 1s - loss: 0.1829 - accuracy: 0.9510 - val_loss: 0.1940 - val_accuracy: 0.9455
Epoch 8/20
60000/60000 - 0s - loss: 0.1797 - accuracy: 0.9516 - val_loss: 0.1915 - val_accuracy: 0.9453
Epoch 9/20
60000/60000 - 0s - loss: 0.1765 - accuracy: 0.9524 - val_loss: 0.1892 - val_accuracy: 0.9463
Epoch 10/20
60

In [8]:
model.save('model.h5')

In [1]:
%%writefile ml_server.py

import json
import tensorflow as tf
import numpy as np
import random

from flask import Flask, request

app = Flask(__name__)
model = tf.keras.models.load_model('model.h5')

feature_model = tf.keras.models.Model(
    model.inputs,
    [layer.output for layer in model.layers]
)

_, (x_test, _) = tf.keras.datasets.mnist.load_data()
x_test = x_test / 255

def get_prediction():
    index = np.random.choice(x_test.shape[0])
    image = x_test[index, :, :]
    image_arr = np.reshape(image, (1, 784))
    return feature_model.predict(image_arr), image

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        preds, image = get_prediction()
        final_preds = [p.tolist() for p in preds]
        return json.dumps({
            'prediction': final_preds,
            'image': image.tolist()
        })
    return 'Welcome to the model server!'
if __name__ == '__main__':
    app.run()

Overwriting ml_server.py


In [13]:
%%writefile app.py

import streamlit as st
import json
import requests
import matplotlib.pyplot as plt
import numpy as np

URI = 'http://127.0.0.1:5000'

st.title('Nural Network Visualizer')
st.sidebar.markdown('## Input Image')

if st.button('Get Random Prediction'):
    response = requests.post(URI, data={})
    response = json.loads(response.text)
    preds = response.get('prediction')
    image = response.get('image')
    image = np.reshape(image, (28, 28))
    
    st.sidebar.image(image, width=150)
    
    for layer, p in enumerate(preds):
        numbers = np.squeeze(np.array(p))
        
        plt.figure(figsize=(32, 4))
        
        if layer == 2:
            row = 1
            col = 10
        else:
            row = 2
            col = 16
        
        for i, number in enumerate(numbers):
            plt.subplot(row, col, i+1)
            plt.imshow(number * np.ones((8, 8, 3)).astype('float32'))
            plt.xticks([])
            plt.yticks([])
            
            if layer == 2:
                plt.xlabel(str(i), fontsize=40)
        plt.subplots_adjust(wspace=0.05, hspace=0.05)
        plt.tight_layout()
        st.text('Layer {}'.format(layer + 1))
        st.pyplot()
            

Overwriting app.py
